In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from dataloader.original import get_dataset
import torch
from torch.utils.data import DataLoader
from architecture.pipe import get_model
from tqdm import tqdm
import os

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
training_data = DataLoader(get_dataset('train', augmentation=True), batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
testing_data = DataLoader(get_dataset('train', augmentation=True), batch_size=32, shuffle=True, num_workers=8, pin_memory=True)

In [6]:
vision_model = get_model()

In [7]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(vision_model.parameters(), lr=1e-4)

In [8]:
state = torch.load('./saves/exp_2/model_epoch_10.pth')
vision_model.load_state_dict(state['model_state_dict'])
optimizer.load_state_dict(state['optimizer_state_dict'])
del state

In [9]:
def train_loop(dataloader, testing_data, model, loss_fn, optimizer, epochs=10, save_path='./saves'):
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    all_exps = [int(elm.replace('exp_', '')) if elm != '.ipynb_checkpoints' else -1 for elm in os.listdir(save_path)]

    current_num = max(all_exps) if (len(all_exps) > 0) else 0
    save_path = save_path + '/exp_' + str(current_num + 1)
    os.mkdir(save_path)

    model.to(device)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        loop = tqdm(dataloader, total=len(dataloader), leave=True)
        total_loss = 0
        accuracy = 0

        for batch_idx, (data, targets) in enumerate(loop):
            data = data.to(device)
            targets = targets.to(device)

            # Forward pass
            scores = model(data)
            loss = loss_fn(scores, targets)
            
            batch_accuracy = ((scores.argmax(-1) == targets.argmax(-1)).int().sum() / len(data)).item()
            accuracy += batch_accuracy

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update progress bar
            loop.set_postfix(loss=loss.item(), accuracy=batch_accuracy)
        
        average_training_accuracy = accuracy.mean()

        model.eval()
        testing_loss = 0
        accuracy = 0
        for test in tqdm(testing_data):
            X, y = test
            
            X, y = X.to(device), y.to(device)
            
            with torch.no_grad():                
                scores = model(X)
                testing_loss =+ loss_fn(scores, y)
                
            accuracy += ((scores.argmax(-1) == y.argmax(-1)).int().sum() / len(X)).item()
            
        average_testing_accuracy = accuracy.mean()

        model.train()

        print(f"Epoch {epoch + 1} average loss: {total_loss / len(dataloader)} with testing loss of {testing_loss / len(testing_data)}, with traning accuracy {average_testing_accuracy}, and testing accuracy {average_testing_accuracy}")

        # Save the models after each epoch
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss / len(dataloader),
            'testing_loss': (testing_loss / len(testing_data)).cpu(),
            'average_training_accuracy': average_training_accuracy.cpu(),
            'average_testing_accuracy': average_testing_accuracy.cpu(),
        }, f"{save_path}/model_epoch_{epoch + 1}.pth")

    print("Training complete!")

In [ ]:
train_loop(training_data, testing_data, vision_model, loss_fn, optimizer, epochs=10)

Epoch 1/10


  0%|          | 0/856 [00:00<?, ?it/s]